#  Introduction to Federated Learning in TF Encrypted

Combining the powerful distributing computing platform provided by TensorFlow with the encrypted computations provided by TF Encrypted allows us to create a flexible federated learning system. Here we also leverage TF 2.0 which gives us even more power to debug, train and prepare models for deployment.

While this is only a simulation and shouldn't be used in production this tutorial will help give you a foundation to continue to explore with federated learning and start thinking about how this could be further enhanced and deployed into a machine learning pipeline.

In [1]:
import tensorflow as tf
import tf_encrypted as tfe

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/justinpatriquin/projects/tf-encrypted/tf_encrypted/operations/secure_random/secure_random_module_tf_2.0.0.so'


To start we must define who our players are. This is only for local computations to help easily explore the power of federated learning. In a further part we'll show how this can be extended to work on actual servers running in the cloud.

For the encrypted computations we need three players (server1, server2 and the crypto-producer). These servers work together to securely compute the mean of the gradients as seen later on.

To host the model and data we need at least two players but to make it more interesting we're using four players. One for the model owner (responible for initial weights, updating the weights and evaluating the model) and three data owners who hold onto their own data and locally calculate the gradients.

In [2]:
players = [
    'server0', 
    'server1', 
    'crypto-producer', 
    'model-owner',
    'data-owner-0',
    'data-owner-1',
    'data-owner-2',
]
config = tfe.EagerLocalConfig(players)

tfe.set_config(config)
tfe.set_protocol(tfe.protocol.Pond())

Here we import some local helper functions. 

`BaseModelOwner` and `BaseDataOwner` will help us run the computations on the correct players and synchronize the communication so that each player is calculating the gradients for the same round. These base classes also allow us to easily extend these computations so that we can customize the computations are run.

The other three functions contain some already implemented functions that we can use to start customizing our federated learning computations.

- `default_model_fn` contains the code that does the forward and backward pass on the model
- `secure_mean` uses encrypted computations to calculate the mean over the gradients
- `evaluate_classifier` evaluates the model and returns the loss

`split_dataset` is a helper function to split the dataset into chunks so that each data owner gets a unique set of the dataset. Its important to note this is only for simulation and doesn't make any sense in the real world.

`download_mnist` is a helper function to save an mnist dataset to disk in the form of tfrecords.

In [3]:
from players import BaseModelOwner, BaseDataOwner
from func_lib import default_model_fn, secure_mean, evaluate_classifier
from util import split_dataset
from download import download_mnist

Here are some globals that help customize the training loop. Note: `NUM_DATA_OWNERS` must match how many data owners are in the above configuration and `DATA_ITEMS` must match how many rows are in the dataset.

In [4]:
NUM_DATA_OWNERS = 3
BATCH_SIZE = 256
DATA_ITEMS = 60000
BATCHES = DATA_ITEMS // NUM_DATA_OWNERS // BATCH_SIZE
LEARNING_RATE = 0.01

Next up is subclassing the `BaseModelOwner` and `BaseDataOwner` classes.

For the `ModelOwner` you can see we override three functions defined as abstract class methods in `BaseModelOwner`. Inside each of these functions we can customize the function that actually needs to be called. Here you can see us place in the functions that we've imported above (`default_model_fn`, `secure_mean` and `evaluate_classifier`).

In [5]:
class ModelOwner(BaseModelOwner):
  @classmethod
  def model_fn(cls, data_owner):
    return default_model_fn(data_owner)

  @classmethod
  def aggregator_fn(cls, model_gradients, model):
    return secure_mean(model_gradients)

  @classmethod
  def evaluator_fn(cls, model_owner):
    return evaluate_classifier(model_owner)

As for `BaseDataOwner` there is no customization needed for right so we leave it empty for now. We'll see how can add some customized function later in the tutorial

In [6]:
class DataOwner(BaseDataOwner):
  pass

To start the main parts of the computation we first need to download the MNIST dataset and split it amongst the parties. The data is downloaded to the directory `./data`.

In [7]:
download_mnist()
split_dataset("./data", NUM_DATA_OWNERS, DATA_ITEMS)

The next section is pretty normal TensorFlow where we define a simple `Sequential` model in Keras that will be able to classify the MNIST dataset. We set the loss and optimizer to variables so that we can pass them into the `ModelOwner` and the `DataOwner`.

In [8]:
model = tf.keras.Sequential((
    tf.keras.layers.Dense(512, input_shape=[None, 28 * 28],
                          activation='relu'),
    tf.keras.layers.Dense(10),
))

model.build()

loss = tf.keras.losses.sparse_categorical_crossentropy
opt = tf.keras.optimizers.Adam(LEARNING_RATE)

Now we need to initialize the model owner and the data owners. Pretty straight-forward but important to note that the names pass into the first arguments must match the names references in the above configuration.

In [9]:
model_owner = ModelOwner("model-owner",
                         "{}/train.tfrecord".format("./data"),
                         model, loss,
                         optimizer=opt)

# Simplify this with a loop?
data_owners = [DataOwner("data-owner-{}".format(i),
                         "{}/train{}.tfrecord".format("./data", i),
                         model, loss,
                         optimizer=opt)
              for i in range(NUM_DATA_OWNERS)]

`.fit` is the main event here. It takes in a few basic arguments and then orchestrators the computation using the cusotmized `ModelOwner` and `DataOwner` functions.

In [10]:
model_owner.fit(data_owners, rounds=BATCHES, evaluate_every=10)

print("\nDone training!!")

69/78 [=========================>....] - ETA: 1s - Loss: 0.1766
Done training!!
